In [1]:
!pip install groq

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.4/105.4 kB 1.7 MB/s eta 0:00:00a 0:00:01


In [10]:
import os

os.environ["GROQ_API_KEY"]=
#"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"

In [ ]:
import json
import re
import pandas as pd

def llm_keyword_extraction(text):
    keyword_prompt = f"""You are an expert at identifying the most important keywords in a text for search engine optimization (SEO). 
    Analyze the following text and extract the top 5 keywords or key phrases that best capture the core topic and are likely to be used in search queries.
    Also, identify if there is a location mentioned in the text. 
    If there is no location, return False. 
    Display the keywords and location in a JSON format with q with single column for keywords and location with single column for the location.
    Your output response should only contain the JSON and not a single more word.
    This is an example of good response: {{'q': ['hot dogs', 'Best hot dogs', 'Paris'], 'location': 'Paris'}}
    Text: {text}
    
    Output:"""

    chat_completion = client.chat.completions.create(
        messages=[{"role": "user", "content": keyword_prompt}],
        model="llama3-8b-8192",
    )
    
    response = chat_completion.choices[0].message.content.strip()
    
    try:
        output = json.loads(response)
    except json.JSONDecodeError:
        json_match = re.search(r'\{.*\}', response, re.DOTALL)
        if json_match:
            json_response = json_match.group(0)
            try:
                output = json.loads(json_response)
            except json.JSONDecodeError:
                print("Failed to decode JSON. Response from LLM:")
                print(response)
                output = {'q': [], 'location': False}
        else:
            print("No JSON object found in response. Response from LLM:")
            print(response)
            output = {'q': [], 'location': False}
    
    return output


text_input = [
        'Find the best restaurants in New York with great ambiance and food',
        'Best hot dogs in Paris',
        'Shaormerii cu specific arabesc in Brasov',
        'Smashed Burgers in cluj la un pret mic',
        'Sandvis cubanez in orasul Buzau',
        'Restaurante cu pizza napoletana si pet friendly',
        'Restaurant cu mancare traditionala romaneasca',
        'Mamaliga cu branza si smantana'
    ]

df = pd.DataFrame()
for text in text_input:
    new_row = {'prompt': text, 'keywords': llm_keyword_extraction(text)}
    df = pd.concat([df, pd.DataFrame([new_row])], ignore_index=True)
print(df)


                                              prompt  \
0  Find the best restaurants in New York with gre...   
1                             Best hot dogs in Paris   
2           Shaormerii cu specific arabesc in Brasov   
3             Smashed Burgers in cluj la un pret mic   
4                    Sandvis cubanez in orasul Buzau   
5    Restaurante cu pizza napoletana si pet friendly   
6      Restaurant cu mancare traditionala romaneasca   
7                     Mamaliga cu branza si smantana   

                                            keywords  
0  {'q': ['best restaurants', 'restaurants in New...  
1  {'q': ['Best hot dogs', 'hot dogs', 'dogs Pari...  
2            {'q': ['Brasov'], 'location': 'Brasov'}  
3  {'q': ['Smashed Burgers', 'Cluj'], 'location':...  
4  {'q': ['cubanez', 'Buzau', 'orasul', 'Buzau', ...  
5  {'q': ['pizza', 'pet friendly', 'napoletana', ...  
6  {'q': ['Romanian food', 'Traditional Romanian ...  
7  {'q': ['mamaliga', 'branza', 'smantana'], 'loc...  


In [30]:
df['keywords'][1]

{'q': ['hot dogs', 'Best hot dogs', 'Paris'], 'location': 'Paris'}